In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords

Парсим данные

In [ ]:
train = pd.read_csv('../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip',header=0,delimiter="\t",quoting=3)
train.head()

In [ ]:
test = pd.read_csv('../input/word2vec-nlp-tutorial/testData.tsv.zip',delimiter='\t',quoting=3,header=0)
test.head()

Очищаем данные от лишних символов

In [ ]:
def review_to_words(raw_review):
    example1 = BeautifulSoup(raw_review,features='lxml')


    letters_only = re.sub("[^a-zA-Z]"," ",example1.get_text())

    lower_case = letters_only.lower()
    words = lower_case.split()


    stops = set(stopwords.words('english'))
    words = [w for w in words if  w not in stops]
    return " ".join(words)

num_reviews = train['review'].size
clean_train_review = []
for i in range(0,num_reviews):
    clean_train_review.append(review_to_words(train['review'][i]))

Собираем вектора признаков

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer='word',preprocessor=None,tokenizer=None,stop_words=None,max_features=5000)
train_data_features = vectorizer.fit_transform(clean_train_review)

train_data_features = train_data_features.toarray()

Строим модель RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf.fit(train_data_features,train['sentiment'])

clean_test_review = []
for i in range(0,test['review'].size):
    clean_test_review.append(review_to_words(test['review'][i]))
test_data_features = vectorizer.transform(clean_test_review).toarray()

Выполняем предсказания

In [ ]:
result = rf.predict(test_data_features)

res = pd.DataFrame({'id':test['id'],'sentiment':result})
res.to_csv("submission.csv", index=False, quoting=3)